# Graphical Lasso

In this part of the notebook we will implement the Graphical Lasso.  The implementation is based upon

Friedman, Jerome, Trevor Hastie, and Robert Tibshirani. "Sparse inverse covariance estimation with the graphical lasso." Biostatistics 9.3 (2008): 432-441.

The idea is to solve the following maximization problem

$$
\arg \max_{\Theta} \log \det \Theta - \text{trace}(S \Theta) - \rho \| \Theta \|_1
$$

where $S$ is an empirical covariance matrix.

In [1]:
import cvxpy as cvx
import numpy as np
import matplotlib.pylab as py
%matplotlib inline

In [2]:
def estimatePhi(S, rho):
    """
        S is the empirical covariance matrix.
    """
    assert S.shape[0] == S.shape[1], "Matrix must be square"
    n = S.shape[0]
    
    Phi = cvx.Variable(n, n)
    
    obj = cvx.Minimize(-(cvx.log_det(Phi) - cvx.trace(S*Phi) - rho*cvx.norm(Phi,1)))
    constraints = []

    prob = cvx.Problem(obj,constraints)
    prob.solve(solver=cvx.SCS, eps=1e-5)
    return Phi.value

We first start with a sparse information matrix.  In other words, an information matrix with almost all $0$ entries.

In [3]:
n = 5
samples = 1000
RInvTrue = np.matrix(np.zeros([n,n]))
RInvTrue += np.eye(n)
RInvTrue[1,2] = 0.5
RInvTrue[2,1] = 0.5

In [4]:
print 'RInvTrue'
print RInvTrue
plt.imshow(prec)
plt.show()
RTrue = np.linalg.inv(RInvTrue)
print 'RTrue'
print RTrue

X = np.matrix(np.random.multivariate_normal(np.zeros([n]),RTrue,samples))
R = X.T*X/samples
print 'R'
print R

RInv = np.linalg.inv(R)
print 'RInv'
print RInv

RInvEstimated = estimatePhi(R,0.1)
print 'RInvEstimated'
print RInvEstimated

RInvTrue
[[ 1.   0.   0.   0.   0. ]
 [ 0.   1.   0.5  0.   0. ]
 [ 0.   0.5  1.   0.   0. ]
 [ 0.   0.   0.   1.   0. ]
 [ 0.   0.   0.   0.   1. ]]
RTrue
[[ 1.          0.          0.          0.          0.        ]
 [ 0.          1.33333333 -0.66666667  0.          0.        ]
 [ 0.         -0.66666667  1.33333333  0.          0.        ]
 [ 0.          0.          0.          1.          0.        ]
 [ 0.          0.          0.          0.          1.        ]]
R
[[  9.83601833e-01  -5.87340489e-02  -5.89734485e-04   2.76711234e-02
    4.28487690e-02]
 [ -5.87340489e-02   1.41148659e+00  -7.48435168e-01  -1.30036715e-02
   -1.35509718e-03]
 [ -5.89734485e-04  -7.48435168e-01   1.42321244e+00   2.99137724e-02
   -2.94709385e-02]
 [  2.76711234e-02  -1.30036715e-02   2.99137724e-02   1.05088199e+00
   -4.11151530e-02]
 [  4.28487690e-02  -1.35509718e-03  -2.94709385e-02  -4.11151530e-02
    9.95386414e-01]]
RInv
[[ 1.02293167  0.0586809   0.03097276 -0.02882108 -0.04422808]
 [ 0.05

Next we try an information matrix with just a few zeros.

In [5]:
n = 5
samples = 1000
np.random.seed(12456)
RInvTrue = np.matrix(np.zeros([n,n]))
for i in range(n):
    for j in range(i,n):
        RInvTrue[i,j] = RInvTrue[j,i] = np.random.normal()

RInvTrue = RInvTrue*RInvTrue.T        
RInvTrue[1,2] = 0
RInvTrue[2,1] = 0
print 'RInvTrue'
print RInvTrue

RInvTrue
[[  3.63026358e+00   5.82619647e+00   9.15587485e-03   5.08954591e-01
   -1.24473743e+00]
 [  5.82619647e+00   1.02233035e+01   0.00000000e+00   3.54054108e-01
   -2.33702158e+00]
 [  9.15587485e-03   0.00000000e+00   7.49555634e+00   2.55037573e+00
   -4.18116602e+00]
 [  5.08954591e-01   3.54054108e-01   2.55037573e+00   2.85996660e+00
   -1.16473124e+00]
 [ -1.24473743e+00  -2.33702158e+00  -4.18116602e+00  -1.16473124e+00
    5.00335538e+00]]


In [6]:
RTrue = np.linalg.inv(RInvTrue)
print 'RTrue'
print RTrue

X = np.matrix(np.random.multivariate_normal(np.zeros([n]),RTrue,samples))
R = X.T*X/samples
print 'R'
print R

RTrue
[[ 3.81001377 -2.16831202  0.14906074 -0.57251319 -0.07365088]
 [-2.16831202  1.3588853  -0.0093787   0.28902267  0.15473316]
 [ 0.14906074 -0.0093787   0.38028366 -0.24496715  0.29346935]
 [-0.57251319  0.28902267 -0.24496715  0.60518593 -0.07126133]
 [-0.07365088  0.15473316  0.29346935 -0.07126133  0.48247273]]
R
[[  3.53219977e+00  -2.01187009e+00   1.33285274e-01  -5.34266617e-01
   -6.65790295e-02]
 [ -2.01187009e+00   1.27467508e+00  -2.72453132e-03   2.68473675e-01
    1.49942818e-01]
 [  1.33285274e-01  -2.72453132e-03   3.82884818e-01  -2.39312311e-01
    3.03079813e-01]
 [ -5.34266617e-01   2.68473675e-01  -2.39312311e-01   5.80348365e-01
   -9.25384333e-02]
 [ -6.65790295e-02   1.49942818e-01   3.03079813e-01  -9.25384333e-02
    4.80896312e-01]]


In [7]:
RInv = np.linalg.inv(R)
print 'RInv'
print RInv

RInvEstimated = estimatePhi(R,0.05)
print 'RInvEstimated'
print RInvEstimated

RInv
[[ 3.50064744  5.56428183  0.08524467  0.49088942 -1.2095418 ]
 [ 5.56428183  9.77835092  0.14641908  0.2901665  -2.31495667]
 [ 0.08524467  0.14641908  7.48497649  2.41371462 -4.2867097 ]
 [ 0.49088942  0.2901665   2.41371462  2.87826187 -0.9898678 ]
 [-1.2095418  -2.31495667 -4.2867097  -0.9898678   5.1449664 ]]
RInvEstimated
[[  1.51957410e+00   2.20244473e+00  -2.16246270e-02   2.55599067e-01
   -2.73727955e-02]
 [  2.20244470e+00   4.03781450e+00   3.76606243e-09  -3.16708988e-08
   -2.76533435e-01]
 [ -2.16246277e-02   3.79452714e-09   3.60552981e+00   8.40733273e-01
   -1.58098420e+00]
 [  2.55599067e-01  -3.15792593e-08   8.40733279e-01   2.03527931e+00
   -1.37300699e-07]
 [ -2.73727921e-02  -2.76533431e-01  -1.58098423e+00  -1.37570212e-07
    2.68320318e+00]]
